**기존 비전 학습의 한계**

- Vision Transformer (ViT, 2020)는 대용량 데이터와 사전학습(pretraining)을 통해 뛰어난 성능을 냈지만,
자기지도학습(self-supervised learning)으로 효율적인 표현을 학습하기는 어려웠음.
 
- SimCLR, MoCo, BYOL 같은 contrastive learning 기법은 이미지 쌍 간의 관계를 학습하지만
    - positive/negative 쌍 설계가 복잡함
    - augmentation 의존도가 높음
    - 계산량이 많음

**배경**

- BERT (2018)는 입력의 일부 토큰을 masking → 복원하는 단순한 자기지도(pretext) task로 뛰어난 성능을 보여줌.
- 저자들은 “이미지에서도 이런 mask reconstruction 방식을 적용할 수 있지 않을까?” 라는 아이디어에서 출발.

핵심 목표

이미지를 일부 가리고(masking), 나머지로부터 원본을 복원하도록 학습하면 좋은 시각 표현을 얻을 수 있지 않을까?

입력의 대부분을 가리는 Masked Image Modeling (MIM)

- 입력 이미지를 ViT patch 단위(예: 16×16)로 나누고 이 중 랜덤하게 75% 정도를 mask.
    
- 나머지 25%의 patch만 인코더에 입력.
- 디코더가 mask된 패치를 픽셀 단위로 복원하도록 학습.



**비대칭 구조 (Asymmetric Encoder–Decoder Architecture)**

- Encoder:
    - 입력의 *일부(25%)*만 처리 → 계산량 절감
    - ViT 구조 (Patch Embedding + Transformer Layers)
- Decoder:
    - mask된 토큰을 포함한 전체 토큰을 입력받음
    - 더 얕고 가벼움 (few Transformer blocks)
    - 원본 이미지를 복원(reconstruction)

**→ Encoder는 representation 학습, Decoder는 복원 전용.**

![mae](F:\mae-from-scratch\notebook\mae.png)

**Reconstruction Target**

- 입력 이미지를 patch 단위로 normalize된 pixel value로 복원
    - 즉, 색상 정보에 집중하지 않고 구조적 패턴에 집중
- (이후 연구에서 feature-level reconstruction도 시도됨 — e.g., iBOT, BEiT 등)

**Pretraining → Finetuning**

1. Pretrain:
    
    Masked reconstruction task로 대규모 데이터(예: ImageNet-1K)에서 학습
    
2. Finetune:
    
    Encoder만 남기고 classification head를 붙여 downstream task 학습